# Baseline model (Modèle de référence)

L'idée c'est de faire tourner un modèle simple afin d'avoir une référence pour comparer nos futurs modèles. A discuter de ce qu'on prend comme modèle de référence.

In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv("data/train.csv", sep= ",")